In [279]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.metrics import accuracy_score
import torch
from torch.optim import SGD, Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset, DataLoader, random_split

In [268]:
df = datasets.load_iris()
# df = pd.DataFrame(df.data, columns=df.feature_names)
# df.values[:, -1]
X, y = df.data, df.target
LabelEncoder().fit_transform(y)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

In [280]:
class IRISDataSet(Dataset):
    def __init__(self):
        df = datasets.load_iris()
        self.X = df.data.astype("float32")
        self.y = df.target.astype("float32")
        # self.y = self.y.reshape((len(self.y), 1))

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

    def get_splits(self, n_test=0.3):
        test_size = round(len(self.X) * n_test)
        train_size = len(self.X) - test_size
        return random_split(self, [train_size, test_size])

test = IRISDataSet()
x1, x2 = test.get_splits()
x1[1]

(array([6.9, 3.1, 4.9, 1.5], dtype=float32), 1.0)

In [281]:
class MLP(torch.nn.Module):
    def __init__(self, n_inputs, n_hidden1, n_hidden2, n_output):
        super().__init__()
        self.hidden1 = torch.nn.Linear(n_inputs, n_hidden1)
        self.act1 = torch.nn.ReLU()
        self.hidden2 = torch.nn.Linear(n_hidden1, n_hidden2)
        self.act2 = torch.nn.ReLU()
        self.hidden3 = torch.nn.Linear(n_hidden2, n_output)
        self.act3 = torch.nn.Softmax(dim=1)
    
    def forward(self, X):
        X = self.hidden1(X)
        X = self.act1(X)
        X = self.hidden2(X)
        X = self.act2(X)
        X = self.hidden3(X)
        X = self.act3(X)

        return X


In [290]:
def prepare_data():
    dataset = IRISDataSet()
    train, test = dataset.get_splits()
    train_dl = DataLoader(train, batch_size=2, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl

In [302]:
def train_model(train_dl, model, loss, trainer, n_epochs):
    criterion = loss
    optimizer = trainer
    for epoch in range(n_epochs):
        for i, (x, y) in enumerate(train_dl):
            y = y.long()
            optimizer.zero_grad()
            y_hat = model(x)
            loss = criterion(y_hat, y)
            loss.backward()
            print(f"epoch: {epoch}, batch: {i}, loss: {loss.data}.")
            optimizer.step()

In [307]:
def evaluate_model(test_dl, model):
    predictions, actuals = [], []
    for i, (x, y) in enumerate(test_dl):
        y_hat = model(x)
        y_hat = y_hat.detach().numpy()
        y = y.numpy()
        y_hat = argmax(y_hat)
        y = y.reshape((len(y), 1))
        y_hat = y_hat.reshape((len(y_hat), 1))
        predictions.append(y_hat)
        actuals.append(y)
    predictions, actuals = np.vstack(predictions), np.vstack(actuals)
    acc = accuracy_score(actuals, predictions)
    return acc

In [304]:
train_dl, test_dl = prepare_data()
print(len(train_dl), len(test_dl))
model = MLP(4, 5, 6, 3)
loss = CrossEntropyLoss()
trainer = Adam(model.parameters())

53 1


In [305]:
train_dl, test_dl = prepare_data()
print(len(train_dl), len(test_dl))
model = MLP(4, 5, 6, 3)
loss = CrossEntropyLoss()
trainer = Adam(model.parameters())
train_model(train_dl, model, loss, trainer, 100)

53 1
epoch: 0, batch: 0, loss: 1.1068472862243652.
epoch: 0, batch: 1, loss: 1.1036326885223389.
epoch: 0, batch: 2, loss: 1.0889215469360352.
epoch: 0, batch: 3, loss: 1.1161584854125977.
epoch: 0, batch: 4, loss: 1.0901367664337158.
epoch: 0, batch: 5, loss: 1.0891005992889404.
epoch: 0, batch: 6, loss: 1.0871601104736328.
epoch: 0, batch: 7, loss: 1.0976239442825317.
epoch: 0, batch: 8, loss: 1.0843257904052734.
epoch: 0, batch: 9, loss: 1.0985708236694336.
epoch: 0, batch: 10, loss: 1.099625587463379.
epoch: 0, batch: 11, loss: 1.0734305381774902.
epoch: 0, batch: 12, loss: 1.0941680669784546.
epoch: 0, batch: 13, loss: 1.0712535381317139.
epoch: 0, batch: 14, loss: 1.0845389366149902.
epoch: 0, batch: 15, loss: 1.0704131126403809.
epoch: 0, batch: 16, loss: 1.095430850982666.
epoch: 0, batch: 17, loss: 1.0795096158981323.
epoch: 0, batch: 18, loss: 1.0879490375518799.
epoch: 0, batch: 19, loss: 1.0819432735443115.
epoch: 0, batch: 20, loss: 1.0833990573883057.
epoch: 0, batch: 21,

In [308]:
acc = evaluate_model(test_dl, model)
acc

TypeError: object of type 'numpy.int64' has no len()

In [212]:
import torch

class MLP(torch.nn.Module):
    def __init__(self, n_inputs, n_hidden1, n_hidden2, n_output):
        super().__init__()
        self.hidden1 = torch.nn.Linear(n_inputs, n_hidden1)
        self.act1 = torch.nn.ReLU()
        self.hidden2 = torch.nn.Linear(n_hidden1, n_hidden2)
        self.act2 = torch.nn.ReLU()
        self.hidden3 = torch.nn.Linear(n_hidden2, n_output)
        self.act3 = torch.nn.ReLU()
    
    def forward(self, X):
        X = self.hidden1(X)
        X = self.act1(X)
        X = self.hidden2(X)
        X = self.act2(X)
        X = self.hidden3(X)
        X = self.act3(X)

        return X

# 初始化MLP
n_inputs = 4   # 假设输入特征数量为4
n_hidden1 = 10
n_hidden2 = 5
n_output = 3
model = MLP(n_inputs, n_hidden1, n_hidden2, n_output)

# # 生成测试数据
# test_input = torch.randn(1, n_inputs)  # 假设我们只生成一个数据点来测试

# # 通过模型传递测试数据
# output = model(test_input)
# print("Output:", output.shape)

# x, y = next(iter(enumerate(train_dl)))
dataset = IRISDataSet()
train, test = dataset.get_splits()
# train_dl = DataLoader(train, batch_size=32, shuffle=True)
# test_dl = DataLoader(test, batch_size=1024, shuffle=False)
# idx, (x, y) = next(enumerate(train_dl))
model(x)


32

In [272]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

# 数据加载和预处理
iris = datasets.load_iris()
X = iris.data
y = iris.target

# 标准化特征以帮助训练
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 转换为torch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.long)  # CrossEntropyLoss要求目标是长整型

# 划分数据集
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# 创建 DataLoader
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=False)

# 定义 MLP 模型
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden1 = nn.Linear(4, 10)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(10, 10)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(10, 3)

    def forward(self, x):
        x = self.hidden1(x)
        x = self.act1(x)
        x = self.hidden2(x)
        x = self.act2(x)
        x = self.output(x)
        return x

# 实例化模型、损失函数和优化器
model = MLP()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 训练模型
def train_model(num_epochs):
    i = 0
    for epoch in range(num_epochs):
        for data, target in train_loader:
            optimizer.zero_grad()
            output = model(data)
            if i == 0:
                print(output, target)
                i += 1
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# 测试模型
def evaluate_model():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    print(f'Accuracy: {100 * correct / total}%')

# 运行训练和评估
train_model(50)
evaluate_model()


tensor([[-0.3248, -0.1896, -0.0985],
        [-0.3486, -0.1853, -0.1203],
        [-0.2957, -0.2077, -0.0680],
        [-0.2322, -0.1958, -0.0174],
        [-0.3106, -0.2028, -0.0866],
        [-0.2308, -0.2088,  0.0064],
        [-0.2473, -0.2335,  0.0477],
        [-0.2898, -0.2113, -0.0525],
        [-0.2712, -0.2105, -0.0379],
        [-0.3516, -0.1929, -0.1244],
        [-0.2299, -0.1767, -0.0507],
        [-0.2962, -0.2039, -0.0653],
        [-0.2307, -0.2030, -0.0076],
        [-0.3206, -0.2021, -0.0951],
        [-0.2353, -0.2026,  0.0161],
        [-0.3477, -0.1798, -0.1199]], grad_fn=<AddmmBackward0>) tensor([1, 1, 1, 0, 1, 0, 0, 2, 2, 2, 0, 1, 0, 1, 0, 1])
Epoch 1, Loss: 1.0230727195739746
Epoch 2, Loss: 0.8724119067192078
Epoch 3, Loss: 0.5938277244567871
Epoch 4, Loss: 0.6242708563804626
Epoch 5, Loss: 0.3450513482093811
Epoch 6, Loss: 0.36853355169296265
Epoch 7, Loss: 0.22045762836933136
Epoch 8, Loss: 0.04926508292555809
Epoch 9, Loss: 0.21979288756847382
Epoch 10, Loss

In [278]:
y1, y2 = torch.tensor([0.2, 0.7, 0.1]), torch.tensor(1)
criterion(y1, y2)

tensor(0.7679)